In [2]:
from problems.HS100 import HS100, CallableClass
import methods.methods as methods
from typing import Tuple, Union, List, Type
from copy import deepcopy
import pandas as pd
import numpy as np
from numpy import number
from scipy.stats.qmc import LatinHypercube as lhs
from scipy.stats.qmc import scale
from smt.surrogate_models import KRG
import random

In [3]:
def sample_sites(problem: dict, n_samples: int, seed=random.randint(1,1000)) -> pd.DataFrame:
    """ Simple wrapper to sample sites
    
    Parameters
    ----------
    problem : dict
        The variables and constraints given

    n_sample: int
        The number of sites to generare

    seed: int
        Seed for the LHS sampling
   
    Returns
    -------
    pd.DataFrame
        The DataFrame of sites
    """

    variables = list(problem["variables"].keys())
    nind = len(variables)
    # Get all the bounds for the variables
    bounds = np.array([problem["variables"][var]["bounds"] for var in variables])
    # Generate the experiment 
    lhs_instance = lhs(
        nind,
        scramble=True,
        strength=1,
        optimization=None,
        seed=seed,
    )
    # Get the experiment
    normalized_array = lhs_instance.random(n_samples)
    # Scale using the bounds
    exp_array = scale(normalized_array, bounds[:, 0], bounds[:, 1])
    # Create the DataFrame
    exp_df = pd.DataFrame(data=exp_array, columns=variables)
    return(exp_df)


In [4]:
def evaluate_sites(local_eval: CallableClass, exp_data: pd.DataFrame, verbose=True):
    """ evaluates sites passed and returns their constraint violation data

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample: int
            The number of sites to generare
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    eps = 1e-6
    local_problem = local_eval.problem()
    local_eval(exp_data)
    my_constraint_calculator = methods.ConstraintCalculator(local_problem)
    exp_data['__conviol__'] = my_constraint_calculator(exp_data)
    exp_data['__State__'] = pd.cut(exp_data['__conviol__'], [-np.inf, eps, 100*eps, np.inf], include_lowest=True, labels=['Feasible', 'Nearly Feasible', "Infeasible"]).astype("str")
    feasible_sites = (exp_data['__State__'] == 'Feasible').sum()
    percentage = feasible_sites/num_sites * 100
    if verbose: 
        print(f"Test evaluator {local_eval.name} with {len(local_problem['variables'])} variables {percentage}% feasible sites")
    return (exp_data)

In [5]:
def advanced_testing(local_eval: CallableClass, num_sites_training: int, num_sites_testing : int, verbose=True) -> pd.DataFrame:
    """ Wrapper to create an experiment, evaluate the passed in function, create a surrogate model, evaluate model

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample_training: int
            The number of sites to train the sm on

        n_sample_testing: int
            The number of sites to test the sm
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    # get training data
    sample_data = sample_sites(local_eval.problem(), num_sites_training)
    exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = exp_data['__conviol__'].to_numpy()

    # create model
    sm = KRG(theta0=[1e-2])
    sm.set_training_values(xt, yt)
    sm.train()

    # get testing data
    eps = 1e-6
    x = sample_sites(local_eval.problem(), num_sites_testing).to_numpy()
    y = data=sm.predict_values(x).flatten()
    feasible_points = pd.DataFrame(data=x[y < eps], columns=variables)
    
    # evaluate model on testing data
    feasible_points = evaluate_sites(local_eval, feasible_points, verbose=verbose)
    return (feasible_points) 
    

In [9]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = nind**2
test_data = advanced_testing(hs100, num_sites, num_sites**2)
test_data

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 49
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  2.1020410
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 2401
   
   Predicting ...
   Predicting - done. Time (sec):  0.0291967
   
   Prediction time/pt. (sec) :  0.0000122
   
Test evaluator hs100 with 7 variables 8.16326530612245% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,-0.272998,1.874401,-1.319155,-9.185419,4.409601,7.685452,-8.533189,8.133122e+04,-268.397091,274.481117,-223.896978,-141.119286,2.660167,Infeasible
1,2.159981,-1.214418,-2.180143,8.067674,9.179856,1.353683,-9.059517,5.992165e+06,-192.924863,224.105333,61.374753,-143.935388,1.452226,Infeasible
2,1.197974,-0.522015,-3.054288,-2.192241,5.872929,-9.802065,-4.882917,4.129829e+05,78.372912,189.958669,-447.372072,-31.248257,4.484621,Infeasible
3,1.979902,0.083704,0.366418,-5.801908,4.816559,-1.960901,9.316398,1.340591e+05,-39.937913,277.165422,201.915637,96.826490,0.039938,Infeasible
4,4.841077,0.819324,-6.113740,6.348183,4.911213,4.525965,-9.397609,1.505808e+05,-100.863974,-129.560631,-114.103109,-283.275331,3.318917,Infeasible
5,-1.376618,-0.343544,-5.168566,-0.991876,-0.679443,-7.227911,6.671967,4.596381e+03,127.798560,25.838636,-32.536213,49.823486,0.325362,Infeasible
6,4.613763,-0.119327,-3.230947,5.738555,-2.866303,-6.310027,-7.288832,9.539467e+03,-29.735807,137.066591,-207.340073,-156.318159,2.596808,Infeasible
7,2.230454,-0.282193,-5.477535,3.198017,-4.562151,2.488555,0.661609,9.206508e+04,104.410156,-40.560710,112.755355,-87.039461,0.960262,Infeasible
8,1.852340,0.276232,-4.631708,0.181958,-3.614577,-2.204334,-0.740977,2.392275e+04,142.692365,49.881235,118.237534,-52.300443,0.523004,Infeasible
9,-2.027302,0.305592,-1.285061,6.799742,1.416446,-5.623464,7.873665,5.199914e+03,-71.989211,273.377214,115.783795,93.033091,0.071989,Infeasible


In [ ]:
# multi-run training
def experiment_3(local_eval, sites_per_run, runs):
    problem = hs100.problem()
    nind = len(problem['variables'])

    # create model
    sm = KRG(theta0=[1e-2], print_prediction=False)
    
    for i in range(runs):
        if i == 0:
            # get first run's data
            sample_data = sample_sites(local_eval.problem(), sites_per_run)
            exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
        variables = list(local_eval.problem()["variables"].keys())
        xt = exp_data[variables].to_numpy()
        yt = exp_data['__conviol__'].to_numpy()
        
        # train model
        sm.set_training_values(xt, yt)
        sm.train()
    
        # get next run's data
        eps = 1e-6
        x = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
        y = data=sm.predict_values(x).flatten()
        while len(y[y < eps]) < sites_per_run:
            # continue generating sites until enough
            x_p = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
            print(x_p)
            y_p = data=sm.predict_values(x_p).flatten()
            x = np.concatenate((x, x_p))
            y = np.concatenate((y, y_p))
        exp_data = pd.DataFrame(data=x[y < eps], columns=variables)
        
        # evaluate model on testing data
        exp_data = evaluate_sites(local_eval, exp_data)
    return exp_data

In [45]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 10
#experiment_3(hs100, num_sites, 3)